In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
#     rotation_range=20, # rotation range
#     width_shift_range=30/img_size, # 30 pixel
#     height_shift_range=30/img_size, # 30 pixel
#     zoom_range=0.2
)


train_datagenerator1 = train_datagen.flow_from_dataframe(
    train,
    directory='../data/input/imgs/train/semi-supervised_imgs/',
    x_col='image',
    y_col='label',
    target_size=(img_size, img_size),
    class_mode='categorical',
    batch_size=10,
#     seed=seed_value,
    shuffle=True
)

train_datagenerator2 = train_datagen.flow_from_dataframe(
    train,
    directory='../data/input/imgs/train/semi-supervised_imgs/',
    x_col='image',
    y_col='label',
    target_size=(img_size, img_size),
    class_mode='categorical',
    batch_size=10,
#     seed=seed_value,
    shuffle=True
)



In [ ]:
# source https://github.com/DevBruce/CutMixImageDataGenerator_For_Keras
import random
class CutMixImageDataGenerator():
    def __init__(self, generator1, generator2, img_size, batch_size):
        self.batch_index = 0
        self.samples = generator1.samples
        self.class_indices = generator1.class_indices
        self.generator1 = generator1
        self.generator2 = generator2
        self.img_size = img_size
        self.batch_size = batch_size

    def reset_index(self):  # Ordering Reset (If Shuffle is True, Shuffle Again)
        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def reset(self):
        self.batch_index = 0
        self.generator1.reset()
        self.generator2.reset()
        self.reset_index()

    def get_steps_per_epoch(self):
        quotient, remainder = divmod(self.samples, self.batch_size)
        return (quotient + 1) if remainder else quotient
    
    def __len__(self):
        self.get_steps_per_epoch()

    def __next__(self):
        if self.batch_index == 0: self.reset()

        crt_idx = self.batch_index * self.batch_size
        if self.samples > crt_idx + self.batch_size:
            self.batch_index += 1
        else:  # If current index over number of samples
            self.batch_index = 0

        reshape_size = self.batch_size
        last_step_start_idx = (self.get_steps_per_epoch()-1) * self.batch_size
        if crt_idx == last_step_start_idx:
            reshape_size = self.samples - last_step_start_idx
            
        X_1, y_1 = self.generator1.next()
        X_2, y_2 = self.generator2.next()
        
        cut_ratio = np.random.beta(a=1, b=1, size=reshape_size)
        cut_ratio = np.clip(cut_ratio, 0.2, 0.8)
        label_ratio = cut_ratio.reshape(reshape_size, 1)
        cut_img = X_2

        X = X_1
        for i in range(reshape_size):
            cut_size = int((self.img_size-1) * cut_ratio[i])
            y1 = random.randint(0, (self.img_size-1) - cut_size)
            x1 = random.randint(0, (self.img_size-1) - cut_size)
            y2 = y1 + cut_size
            x2 = x1 + cut_size
            cut_arr = cut_img[i][y1:y2, x1:x2]
            cutmix_img = X_1[i]
            cutmix_img[y1:y2, x1:x2] = cut_arr
            X[i] = cutmix_img
            
        y = y_1 * (1 - (label_ratio ** 2)) + y_2 * (label_ratio ** 2)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)

In [ ]:
# !! Define CutMixImageDataGenerator !!
train_generator2 = CutMixImageDataGenerator(
    generator1=train_datagenerator1,
    generator2=train_datagenerator2,
    img_size=224,
    batch_size=10,
)

In [ ]:
hoge  = np.reshape(X[5], (224,224,3))
#貼り付け
plt.imshow(hoge)
#表示
plt.show()